## SQL environment setup

In [1]:
%load_ext sql

In [2]:
%sql sqlite://

### Load data

We're taking data from the CSV and putting it in an in-memory [SQLite](https://sqlite.org/index.html) database via [pandas](https://pandas.pydata.org/).

In [3]:
import pandas as pd

requests = pd.read_csv("311_jan_2022.csv")

In [4]:
%sql --persist requests

 * sqlite://


'Persisted requests'

Ensure the records were loaded:

In [5]:
%sql SELECT COUNT(*) FROM requests LIMIT 10;

 * sqlite://
Done.


COUNT(*)
45863


[Display the schema](https://sqlite.org/cli.html#querying_the_database_schema):

In [6]:
%sql SELECT sql FROM sqlite_schema ORDER BY tbl_name, type DESC, name;

 * sqlite://
Done.


sql
"CREATE TABLE requests ( ""index"" BIGINT, ""Unique Key"" BIGINT, ""Created Date"" TEXT, ""Closed Date"" TEXT, ""Agency"" TEXT, ""Agency Name"" TEXT, ""Complaint Type"" TEXT, ""Descriptor"" TEXT, ""Location Type"" TEXT, ""Incident Zip"" FLOAT, ""Incident Address"" TEXT, ""Street Name"" TEXT, ""Cross Street 1"" TEXT, ""Cross Street 2"" TEXT, ""Intersection Street 1"" TEXT, ""Intersection Street 2"" TEXT, ""Address Type"" TEXT, ""City"" TEXT, ""Landmark"" TEXT, ""Facility Type"" TEXT, ""Status"" TEXT, ""Due Date"" TEXT, ""Resolution Description"" TEXT, ""Resolution Action Updated Date"" TEXT, ""Community Board"" TEXT, ""BBL"" FLOAT, ""Borough"" TEXT, ""X Coordinate (State Plane)"" FLOAT, ""Y Coordinate (State Plane)"" FLOAT, ""Open Data Channel Type"" TEXT, ""Park Facility Name"" TEXT, ""Park Borough"" TEXT, ""Vehicle Type"" TEXT, ""Taxi Company Borough"" TEXT, ""Taxi Pick Up Location"" TEXT, ""Bridge Highway Name"" TEXT, ""Bridge Highway Direction"" TEXT, ""Road Ramp"" TEXT, ""Bridge Highway Segment"" TEXT, ""Latitude"" FLOAT, ""Longitude"" FLOAT, ""Location"" TEXT)"
"CREATE INDEX ix_requests_index ON requests (""index"")"
